In [11]:
import pandas as pd
import numpy as np
import math
import json
import os

# ATTENTION : under windows split on "\\", under linux split on "/"
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("..")
print(os.getcwd())

from matplotlib import pyplot as plt
import seaborn as sns

import utils

/home/lola/Documents/Études/EPFL/MA2/Data Visualization/MigrationViz


### TO DO
- Changer LATIN AMERICA AND THE CARIBBEAN to LATIN AMERICA dans colonnes et lignes
- 


In [12]:
# Initialization
PATH_WORLD = "data_world/"
SEX_DESTINATION_ORIGIN = PATH_WORLD + "undesa_pd_2020_ims_stock_by_sex_destination_and_origin.xlsx"

# Continents and subregions names
CONTINENT = ["AFRICA", "ASIA", "EUROPE", "LATIN AMERICA AND THE CARIBBEAN", "NORTHERN AMERICA", "OCEANIA", "OTHER"]
COMMON_COLUMN = ["destination_region"]
SUBREGION = ['Eastern Africa', 'Middle Africa', 'Northern Africa', 'Southern Africa', 'Western Africa', 'Central Asia', 'Eastern Asia', 'South-Eastern Asia', 'Southern Asia', 'Western Asia', 'Eastern Europe', 'Northern Europe', 'Southern Europe', 'Western Europe', 'Caribbean', 'Central America', 'South America', 'Australia and New Zealand', 'Melanesia', 'NORTHERN AMERICA', 'OCEANIA', 'OTHER']

In [13]:
def csv_to_json_sankey(doc, sheet, region, json_name, json_path = './website/data/'):

    # import table
    sankey = pd.read_excel(doc, sheet_name=sheet, index_col=0, skiprows=9, na_values='..')

    # Keep only lines of interest
    sankey = sankey[22:]

    # Drop columns of no interest (keep only continent and sub-continent)
    column_index = [1, 2, 3] + [x for x in range(4,24)]
    columns_to_drop = sankey.columns[column_index]
    sankey.drop(axis='columns', columns=columns_to_drop, inplace=True)

    # Reset index 
    sankey.reset_index(drop=True, inplace=True)

    # Remove .1 and * details
    sankey.columns = sankey.columns.str.replace(r'[*\.1]', '', regex=True)

    # Remove stupid space in front of names in columns 
    sankey.columns = [col.strip() for col in sankey.columns]

    # Rename column
    sankey.rename(columns={"Region, development group of destination": "destination_region",
                           " LATIN AMERICA AND THE CARIBBEAN ":"LATIN AMERICA"}, inplace=True)
    
    # Remove stupid space in front of names in rows 
    sankey["destination_region"] = sankey.destination_region.str.replace("*","")
    sankey["destination_region"] = sankey["destination_region"].str.strip()

    sankey_region = sankey.copy(deep=True)
    sankey_region = sankey_region[COMMON_COLUMN + region]

    filt_continent = sankey_region["destination_region"].isin(region)
    sankey_region = sankey_region[filt_continent]
    display(sankey_region)

    # Melt dataframe to convert it into long format
    sankey_region_melted = sankey_region.melt(id_vars='destination_region', var_name='source', value_name='value')

    # Create indices for the nodes
    nodes_origin = sorted(set(sankey_region_melted["source"]))
    nodes_source_indices = {node: idx for idx, node in enumerate(nodes_origin)}

    nodes_destination = sorted(set(sankey_region_melted["destination_region"]))
    nodes_target_indices = {node: idx + 7 for idx, node in enumerate(nodes_destination)}

    total_nodes = sorted(nodes_origin) + sorted(nodes_destination)


    # List of nodes
    nodes_list = [{"node": idx, "name": node} for idx, node in enumerate(total_nodes)]

    # List of links
    links_list = [{"source": nodes_source_indices[row["source"]] , "target": nodes_target_indices[row["destination_region"]], "value": row["value"]} for _, row in sankey_region_melted.iterrows()]

    sankey_region_json = json.dumps({
        "nodes": nodes_list, 
        "links": links_list})

    # Export the json data to a document
    path = json_path + json_name
    with open(path, 'w') as f:
        json.dump(sankey_region_json, f, )

    print('JSON file' + path + ' saved successfully ')

In [14]:
# Northern america is both a continent and a subregion, it must be present in both continent list and sub_continent 
# Due to incomplete numbers for other sub_continents of Oceania except Australia & New Zealand, Oceania will be used both as a sub_continent and a continent
# OTHER, representing 12 millions people world-wide, needs to be taken into account every where. 
# TODO Make sure it does not cause problems later number wise 




In [15]:
csv_to_json_sankey(SEX_DESTINATION_ORIGIN, "Table 2", CONTINENT, json_name='sankey_2020.json')

,destination_region,AFRICA,ASIA,EUROPE,LATIN AMERICA AND THE CARIBBEAN,NORTHERN AMERICA,OCEANIA,OTHER
0,AFRICA,20917565,1207631,648455,32524,53563,14483,2515243
6,ASIA,4720103,68497762,7169630,414658,538199,101725,4176425
12,EUROPE,11024274,23203976,44246425,5395924,1100304,397036,1338129
17,LATIN AMERICA AND THE CARIBBEAN,48791,402369,1355886,11297173,1293053,5630,391721
21,NORTHERN AMERICA,3268757,17549235,6869872,25535633,1088520,343625,4053153
22,OCEANIA,587673,4050511,2983395,214569,254319,1107706,182480


JSON file./website/data/sankey_2020.json saved successfully 


In [17]:
csv_to_json_sankey(SEX_DESTINATION_ORIGIN, "Table 3", CONTINENT, json_name='sankey_2015.json')

,destination_region,AFRICA,ASIA,EUROPE,LATIN AMERICA AND THE CARIBBEAN,NORTHERN AMERICA,OCEANIA,OTHER
0,AFRICA,18200172,1078578,581876,29928,47055,12521,2910662
6,ASIA,4166307,61654516,6573735,376798,496804,92431,3830658
12,EUROPE,9536606,19797557,38418244,4487571,972405,348242,1198458
17,LATIN AMERICA AND THE CARIBBEAN,39533,341807,1291715,6166726,1182133,4723,414866
21,NORTHERN AMERICA,2663579,17273538,6926249,24980329,1100000,280255,2409791
22,OCEANIA,505762,3232023,2785934,164648,216252,1013250,154407


JSON file./website/data/sankey_2015.json saved successfully 


In [18]:
csv_to_json_sankey(SEX_DESTINATION_ORIGIN, "Table 4", CONTINENT, json_name='sankey_2010.json')

,destination_region,AFRICA,ASIA,EUROPE,LATIN AMERICA AND THE CARIBBEAN,NORTHERN AMERICA,OCEANIA,OTHER
0,AFRICA,14571247,896987,604211,19077,46499,12023,1656633
6,ASIA,3426719,51825389,6444826,426361,468106,88244,3443995
12,EUROPE,8748280,18287405,36683624,4581301,921685,323142,1081723
17,LATIN AMERICA AND THE CARIBBEAN,27847,285869,1147896,5344184,1198308,4249,318235
21,NORTHERN AMERICA,1984391,14250937,7081741,24134573,1149471,245006,2124405
22,OCEANIA,435368,2575590,2749003,132154,185734,918504,132245


JSON file./website/data/sankey_2010.json saved successfully 


In [19]:
csv_to_json_sankey(SEX_DESTINATION_ORIGIN, "Table 5", CONTINENT, json_name='sankey_2005.json')

,destination_region,AFRICA,ASIA,EUROPE,LATIN AMERICA AND THE CARIBBEAN,NORTHERN AMERICA,OCEANIA,OTHER
0,AFRICA,13411177,789396,617032,10748,45155,8018,1158561
6,ASIA,2335304,40867887,6408006,394800,382401,70300,2791089
12,EUROPE,7620545,16825888,33415975,3453004,824591,284743,1160985
17,LATIN AMERICA AND THE CARIBBEAN,21850,264943,1248692,4357513,967044,3585,320486
21,NORTHERN AMERICA,1524859,12195799,6988925,21019256,1132443,230353,2271454
22,OCEANIA,327671,1862130,2658354,102885,160472,794419,118090


JSON file./website/data/sankey_2005.json saved successfully 


In [20]:
csv_to_json_sankey(SEX_DESTINATION_ORIGIN, "Table 6", CONTINENT, json_name='sankey_2000.json')

,destination_region,AFRICA,ASIA,EUROPE,LATIN AMERICA AND THE CARIBBEAN,NORTHERN AMERICA,OCEANIA,OTHER
0,AFRICA,12753079,681816,533653,9501,40167,6457,1027004
6,ASIA,2164626,36924433,6445653,407595,324065,58045,2742569
12,EUROPE,5797189,15541563,31590963,1699981,735943,259848,1233306
17,LATIN AMERICA AND THE CARIBBEAN,20285,258933,1396387,3777576,796397,3400,286760
21,NORTHERN AMERICA,1115964,10466666,7126994,18644817,1113881,217378,1666010
22,OCEANIA,228860,1460202,2634134,89230,139466,701708,108081


JSON file./website/data/sankey_2000.json saved successfully 


In [21]:
csv_to_json_sankey(SEX_DESTINATION_ORIGIN, "Table 7", CONTINENT, json_name='sankey_1995.json')

,destination_region,AFRICA,ASIA,EUROPE,LATIN AMERICA AND THE CARIBBEAN,NORTHERN AMERICA,OCEANIA,OTHER
0,AFRICA,14179992,567843,539020,9417,47102,7074,1006629
6,ASIA,1953253,34096664,6995360,291724,277759,47408,2755876
12,EUROPE,5179831,14667430,29988843,1340812,743008,222969,1346934
17,LATIN AMERICA AND THE CARIBBEAN,19478,258892,1600698,3670383,688958,2912,420232
21,NORTHERN AMERICA,826822,8290114,6859869,14269456,1054522,175797,1864368
22,OCEANIA,182018,1266979,2652213,87912,134812,615135,83458


JSON file./website/data/sankey_1995.json saved successfully 


In [22]:
csv_to_json_sankey(SEX_DESTINATION_ORIGIN, "Table 8", CONTINENT, json_name='sankey_1900.json')

,destination_region,AFRICA,ASIA,EUROPE,LATIN AMERICA AND THE CARIBBEAN,NORTHERN AMERICA,OCEANIA,OTHER
0,AFRICA,13431488,492107,622122,10188,58134,9090,1066537
6,ASIA,1777604,35481900,7632934,118597,239176,43560,2916178
12,EUROPE,4682594,13453956,28367362,1104536,769469,191405,1038903
17,LATIN AMERICA AND THE CARIBBEAN,19192,263267,1817209,3980808,602762,2552,450181
21,NORTHERN AMERICA,548628,6181878,6645283,9976169,999977,135857,3122616
22,OCEANIA,161958,1083892,2645368,83101,132617,579083,45919


JSON file./website/data/sankey_1900.json saved successfully 
